## Seleção de Features

Neste tópico abordaremos a seleção de features, utilizando o método Boruta, teste de Chi2 e Correlação point-biserial para tal.


Teste de correlação ponto-bisserial é simplesmente a correlação entre uma variável dicotômica e uma variável contínua. Acontece que este é um caso especial da correlação de Pearson. Portanto, calcular a correlação bisserial de ponto especial é equivalente a calcular a correlação de Pearson quando uma variável é dicotômica e a outra é contínua.

Chi2*: Método utilizado para seleção de features categóricas ordinais e nominais. É um método que avalia a indepedência das features em relação ao target.

Boruta: Método muito robusto e baseado em modelo, que leva em consideração a interação entre a features, algo nao levado em conta pelo Chi2 ou outros métodos univariados de seleção. 

Para mais informações sobre o Boruta: https://towardsdatascience.com/boruta-explained-the-way-i-wish-someone-explained-it-to-me-4489d70e154a

## Depêndencias

In [1]:
#Importando bibliotecas
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cloudpickle
import matplotlib.pyplot as plt
from scipy.stats import norm
from pandas.api.types import is_numeric_dtype
from src.selecao_de_features import boruta_selector,chi_squared,point_biserial
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

## Dados e Constantes

In [2]:
DATA_RAW_PATH = os.path.join('..','data','raw')
DATA_RAW_NAME = 'diabetes_data.csv'
DATA_INTER_PATH = os.path.join('..','data','inter')
DATA_INTER_TEST_NAME = 'test_inter.csv'
DATA_INTER_TRAIN_NAME = 'train_inter.csv'
DATA_INTER_TRAIN_FT_SELECTION = 'train_inter_ft_selection.csv'
META_DATA_PATH = os.path.join(DATA_RAW_PATH,'dicionario_tipos.pkl')
TARGET = 'target'

In [3]:
#Dados do dicionário
with open(META_DATA_PATH,'rb') as f:
    dicionario_tipo = cloudpickle.load(f)
df_train = pd.read_csv(os.path.join(DATA_INTER_PATH, DATA_INTER_TRAIN_NAME), dtype=dicionario_tipo)

In [4]:
df_train.head()

,idade,genero,polyuria,polydipsia,perda_de_peso_repentina,fraqueza,polyphagia,candidiase_genital,desfoque_visual,coceira,irritabilidade,cicatrizacao_retardada,paresia_parcial,rigidez_muscular,alopecia,obesidade,target
0,67,male,no,no,no,no,yes,no,yes,no,no,no,no,yes,no,no,negative
1,55,female,yes,yes,yes,no,yes,no,no,yes,no,yes,yes,no,no,no,positive
2,60,male,no,no,no,yes,no,no,no,no,no,no,yes,yes,no,no,negative
3,48,female,yes,yes,no,yes,no,no,yes,yes,no,yes,yes,no,no,no,positive
4,31,female,yes,yes,yes,yes,yes,no,no,yes,yes,no,yes,yes,no,no,positive


## Iniciando análises para seleção

In [5]:
#Utilizando apenas as variaveis numericas.
todas_as_variaveis = set(df_train.columns.tolist())
variaveis_categoricas = set(df_train.select_dtypes(include=['object']).columns.tolist())
variaveis_numericas = todas_as_variaveis - variaveis_categoricas
variaveis_categoricas = list(variaveis_categoricas)

In [6]:
# Criando cópia do dataFrame
df_train_copy = df_train.copy()

## Abordagem (Correlação+Chi2+Boruta)

## Correlação

A Correlação não descartou nenhuma variável. 

In [7]:
#Plot da distribuição da variável target.
df_train['target'] = df_train['target'].map({'positive': 1, 'negative': 0})
df_train['genero'] = df_train['genero'].map({'female':0, 'male': 1})

In [8]:
list_yes_no = ['polyuria', 'polydipsia', 'perda_de_peso_repentina', 'fraqueza', 'polyphagia', 'candidiase_genital', 'desfoque_visual', 'coceira', 'irritabilidade', 'cicatrizacao_retardada', 'paresia_parcial', 'rigidez_muscular', 'alopecia', 'obesidade']
for lista in list_yes_no:
    df_train[lista] = df_train[lista].map({"no":0, "yes":1}.get)

In [9]:
pb_df, columns_remove_pb = point_biserial(df_train, 'target', ['idade'])

In [10]:
pb_df

,column,correlation,p_value,result
0,idade,0.135992,0.005465,Reject H0


In [11]:
columns_remove_pb

[]

O test point biserial não eliminou a feature idade.

## Chi2

In [12]:
chi2_df, logs = chi_squared(df_train_copy, y ='target', cols = variaveis_categoricas)

In [14]:
chi2_df

,column,p-value,chi2_value
0,polyphagia,2.445415e-14,58.136541
1,coceira,7.858988e-01,0.073788
2,genero,1.366399e-21,91.099301
3,perda_de_peso_repentina,1.692650e-20,86.120894
4,paresia_parcial,4.934038e-22,93.115079
5,obesidade,2.908504e-02,4.762573
6,alopecia,7.310311e-08,28.980698
7,irritabilidade,1.394289e-08,32.195282
8,fraqueza,3.977307e-06,21.275753
9,target,NaN,NaN


In [15]:
logs

['A coluna target não pode ser avaliada. ']

Com é o próprio target, não calculamos o chi2 para ele.
* Hipótese nula - H0: As variáveis são independente. 
- Segundo o teste de hipotése realizado, as variáveis cujo p-valor < 0.05 tem relação com o target.

In [16]:
#As variáveis que serão excluídas. 
lista_descart_chi2 = chi2_df[chi2_df['p-value']>=0.05]['column'].tolist()

In [17]:
lista_descart_chi2

['coceira', 'cicatrizacao_retardada']

As variáveis que foram descartas pelo teste de Chi-2 foram, `cicatrização_retardada` e `coceira`, logo mais elas serão descartadas para a modelagem. 

## Boruta

In [18]:
#Vamos filtrar as Features removidas no step anterior antes de passarmos para o Boruta
list_to_drop_boruta = boruta_selector(df_train_copy.drop(lista_descart_chi2,axis=1), y='target')

In [19]:
#Lista de dropagem Geral (chi2 + boruta)
lista_drop_geral = set(lista_descart_chi2+list_to_drop_boruta+columns_remove_pb)
lista_drop_geral

{'candidiase_genital',
 'cicatrizacao_retardada',
 'coceira',
 'desfoque_visual',
 'fraqueza',
 'obesidade',
 'rigidez_muscular'}

In [20]:
# dropando as colunas pela abordagem.
df_train_copy.drop(lista_drop_geral,axis=1).to_csv(os.path.join(DATA_INTER_PATH,DATA_INTER_TRAIN_FT_SELECTION), index=False)

A abordagem para seleção de features que utilizamos são (Chi2 e Boruta), que combinados trazem resultados bastante relevantes, descartando as variáveis que possuem menos relação com target e que consequentemente teriam um baixo poder preditivo. As features descartadas no boruta e Chi2   foram:`candidíase_genital`,`cicatrização_retardada`,`coceira`,`desfoque_visual`,`fraqueza`,`obesidade`, `regidez_muscular`. Todas essas features não serão consideradas para nosso modelo.